In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import torch
from music21 import note, chord, instrument, stream

In [2]:
def create_midi(prediction_output):
        
    offset = 0
    output_notes = []

            # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
                # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
                # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        

                # increase offset each iteration so that notes do not stack
        offset += 0.25

    return output_notes

In [3]:
all_notes_path = "../data/notes_final.pkl"
int_to_note_path = "../data/int_to_note.pkl"

with open(all_notes_path, "rb") as f:
    all_notes = pickle.load(f)


with open(int_to_note_path, "rb") as f:
    int_to_note = pickle.load(f)

In [4]:
### create frequency table
meta = {"Note" : [], "Type" : [], "Count" : []}
for k, v in int_to_note.items():

    meta["Note"].append(v)
    #meta["No."].append(k) -> No. = index of the df

    meta["Count"].append(all_notes.count(v))

    if ('.' in v) or v.isdigit():
        meta["Type"].append("Chord")

    else:
        meta["Type"].append("Note")



df = pd.DataFrame(meta)
df["Prob"] = df["Count"] / df["Count"].sum()

In [5]:
random_generated = torch.multinomial(torch.tensor(df["Prob"].values), 200, replacement=True).tolist()
random_music = [int_to_note[note] for note in random_generated]
midi_stream = stream.Stream(create_midi(random_music))
midi_stream.write("midi", fp="random_02.mid")

'random_02.mid'